# Zero-Shot Baseline Inference

**Introduction**:

The purpose of this notebook is to load the Qwen-2.5-VL-7B-Instruct model from HuggingFace and perform zero-shot inference on the W2 dataset prepared from notebook 1.

Steps: 
  - Bootstrap the environment
  - Load the model
  - Define the system and user prompts
  - Execute baseline inference testing
  - Save the results to the `reports/baseline` directory

Follow the project README for more info on running this notebook. 

# Boostrap environment

In [1]:
# Set working directory
import os
os.environ["APP_PROJECT_DIR"] = "/content/ai-image-to-text"  # override with project directory
os.chdir(os.environ["APP_PROJECT_DIR"])

# Install packages and bootstrap environment
%pip install -q python-dotenv
from src.utils.env_setup import setup_environment
env = setup_environment()
%pip install -q -r requirements-{env}.txt

Loaded application properties from: /content/ai-image-to-text/.env.colab
Working directory: /content/ai-image-to-text
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import libraries
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from huggingface_hub import login as hf_login
import torch
import pandas as pd
import os
from src.utils import data_loader
from src.model import reporting
from src.model.executor import Executor

# Connect to huggingface
hf_login(os.environ["APP_HF_TOKEN"])

# File and directory paths
base_dir = os.environ["APP_PROJECT_DIR"]
datasets_dir = os.environ["APP_DATA_DIR"]
output_dir = os.environ["APP_OUTPUT_DIR"]
dataset_w2s_dir = f"{datasets_dir}/w2s"
dataset_processed_dir = f"{dataset_w2s_dir}/processed"
dataset_processed_final_dir = f"{dataset_processed_dir}/final"
output_results_dir = f"{output_dir}/baseline"
output_results_file = f"{output_results_dir}/results.csv"
output_report_file = f"{output_results_dir}/results_report.txt"
output_report_ADP1_file = f"{output_results_dir}/results_report_ADP1.txt"
output_report_ADP2_file = f"{output_results_dir}/results_report_ADP2.txt"
output_report_IRS1_file = f"{output_results_dir}/results_report_IRS1.txt"
output_report_IRS2_file = f"{output_results_dir}/results_report_IRS2.txt"
system_prompt_file_path = f"{base_dir}/config/system_prompt.txt"
user_prompt_file_path = f"{base_dir}/config/user_prompt.txt"

# general constants
batch_size = 2
max_new_tokens = 256

# Load the model

In [ ]:
# Load model
model_id = "Qwen/Qwen2.5-VL-7B-Instruct"
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
).eval()  # Since this model will only be used for inference

# Load processor
min_pixels = 256 * 28 * 28
max_pixels = 1280 * 28 * 28
processor = AutoProcessor.from_pretrained(
    model_id, min_pixels=min_pixels, max_pixels=max_pixels
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


# Define the prompts

In [4]:
# load system prompt
system_prompt = data_loader.get_text(system_prompt_file_path)
print(system_prompt)

# load user prompt
user_prompt = data_loader.get_text(user_prompt_file_path)
print(f"\n{user_prompt}")

You are an expert in processing W-2 forms. Your task is to extract specific information from the 
authoritative W-2 form in the provided image and present it in a structured JSON object. If the 
image contains multiple forms, the authoritative form is always located in the upper left portion 
of the image. Extract data only from this form, ignoring any duplicates. Use the standard box numbers 
to locate the fields: 

    - Employee Name (Box e),
    - Employer Name (Box c), 
    - Wages and Tips (Box 1), 
    - Federal Income Tax Withheld (Box 2), 
    - Social Security Wages (Box 3), 
    - Medicare Wages and Tips (Box 5), 
    - State (Box 15)
    - State Wages (Box 16)
    - State Income Tax Withheld (Box 17)

For state information, multiple states may be listed. Do not use information from Boxes c or e/f or 
any other areas of the image for state data. 

If a field is missing or blank, use an empty string as the value. Return only the completed JSON object 
without additional comme

# Execute zero shot baseline testing

In [5]:
# Load ground truth data
metadata = data_loader.get_metadata(
    f"{dataset_processed_final_dir}/test/metadata.jsonl",
    f"{dataset_processed_final_dir}/test",
)
print(f"Selected {len(metadata)} ground truth examples for testing.")

# Run test
executor = Executor(
    model=model,
    processor=processor,
    system_prompt=system_prompt,
    user_prompt=user_prompt,
)
df = executor.execute_inference_test(metadata, batch_size, max_new_tokens)

# Save comparison results to CSV
os.makedirs(output_results_dir, exist_ok=True)
df.to_csv(output_results_file, index=False)

Selected 100 ground truth examples for testing.
Processing 100 examples...
Processing batch (1 of 50); batch size = 2.
Processing batch (2 of 50); batch size = 2.
Processing batch (3 of 50); batch size = 2.
Processing batch (4 of 50); batch size = 2.
Processing batch (5 of 50); batch size = 2.
Processing batch (6 of 50); batch size = 2.
Processing batch (7 of 50); batch size = 2.
Processing batch (8 of 50); batch size = 2.
Processing batch (9 of 50); batch size = 2.
Processing batch (10 of 50); batch size = 2.
Processing batch (11 of 50); batch size = 2.
Processing batch (12 of 50); batch size = 2.
Processing batch (13 of 50); batch size = 2.
Processing batch (14 of 50); batch size = 2.
Processing batch (15 of 50); batch size = 2.
Processing batch (16 of 50); batch size = 2.
Processing batch (17 of 50); batch size = 2.
Processing batch (18 of 50); batch size = 2.
Processing batch (19 of 50); batch size = 2.
Processing batch (20 of 50); batch size = 2.
Processing batch (21 of 50); batch

Report the results. Written to report files and standard out.

An additional report is generated for each form type, providing a detailed breakdown by type.

In [6]:
# Read from persisted CSV
df = pd.read_csv(output_results_file)

# output main report - to file and std out
reporting.output_results(df, output_report_file)

# output report for each form type - to file only
form_types = [
    ("ADP1", output_report_ADP1_file),
    ("ADP2", output_report_ADP2_file),
    ("IRS1", output_report_IRS1_file),
    ("IRS2", output_report_IRS2_file),
]
for form_type, report_file_path in form_types:
    reporting.output_results_by_form_type(df, report_file_path, form_type)

**Overall Accuracy**: 87.23%

**Field Summary**:
| Field                       |   total_comparisons |   matches |   mismatches |   accuracy |   mismatch_percentage |
|:----------------------------|--------------------:|----------:|-------------:|-----------:|----------------------:|
| Employee Name               |                 100 |       100 |            0 |       1    |               0       |
| Employer Name               |                 100 |        97 |            3 |       0.97 |               1.80723 |
| Federal Income Tax Withheld |                 100 |        96 |            4 |       0.96 |               2.40964 |
| Field Count Check           |                 100 |       100 |            0 |       1    |               0       |
| Medicare Wages and Tips     |                 100 |        62 |           38 |       0.62 |              22.8916  |
| Social Security Wages       |                 100 |        75 |           25 |       0.75 |              15.0602  |
| State